## Foundational Models

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_generator(latent_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=latent_dim),
        Dense(256, activation='relu'),
        Dense(784, activation='tanh') 
    ])
    return model
    
def build_discriminator(input_shape):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_shape,)),  
        Dense(128, activation='relu'), 
        Dense(1, activation='sigmoid') 
    ])
    return model
    
latent_dim = 100  
generator = build_generator(latent_dim)
discriminator = build_discriminator(784)  

In [ ]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import Model, backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon
    
inputs = Input(shape=(784,)) # Encoder
h = Dense(256, activation='relu')(inputs)
z_mean = Dense(2)(h)
z_log_var = Dense(2)(h)
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])
decoder_h = Dense(256, activation='relu') 
decoder_mean = Dense(784, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
vae = Model(inputs, x_decoded_mean)

In [ ]:
from tensorflow.keras.layers import LSTM

sequence_length = 10  
num_features = 5
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, num_features)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
from tensorflow.keras.layers import Embedding, LayerNormalization, MultiHeadAttention 

def simplified_gpt_model(vocab_size=10000, embed_dim=256, max_length=40, num_heads=4, ff_dim=512):
    inputs = Input(shape=(max_length,)) 
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output + embedding_layer)
    ff_network = Dense(ff_dim, activation="relu")(attn_output)
    ff_network_output = Dense(embed_dim)(ff_network)
    sequence_output = LayerNormalization(epsilon=1e-6)(ff_network_output + attn_output)
    outputs = Dense(vocab_size, activation="softmax")(sequence_output)
    model = Model(inputs=inputs, outputs=outputs)
    return model
    
gpt_model = simplified_gpt_model()
gpt_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

## Semantic Proximity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A, B = np.array([2, 2]), np.array([3, 0])
cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
angle = np.arccos(cos_sim)
plt.figure(figsize=(6,6))
for vector, color, label in zip([A, B], ['red', 'green'], ['Vector A', 'Vector B']):
    plt.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color=color, label=label)
plt.plot(0.5 * np.cos(np.linspace(0, angle, 100)), 0.5 * np.sin(np.linspace(0, angle, 100)), color='blue', label=f'Cosine Similarity = {cos_sim:.2f}')
plt.axis([-1, 4, -1, 4])
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)
plt.title('Cosine Similarity between Vectors')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
sentence_1 = "He turned on the light to read."
sentence_2 = "The light fabric was perfect for summer."
tokens_1 = tokenizer(sentence_1, return_tensors="tf")
tokens_2 = tokenizer(sentence_2, return_tensors="tf")
outputs_1 = model(tokens_1)
outputs_2 = model(tokens_2)
light_index_1 = tokens_1['input_ids'][0].numpy().tolist().index(tokenizer.convert_tokens_to_ids('light'))
light_index_2 = tokens_2['input_ids'][0].numpy().tolist().index(tokenizer.convert_tokens_to_ids('light'))
embedding_1 = outputs_1.last_hidden_state[0, light_index_1]
embedding_2 = outputs_2.last_hidden_state[0, light_index_2]
cosine_similarity = tf.keras.losses.cosine_similarity(embedding_1, embedding_2, axis=0)
cosine_similarity = -cosine_similarity.numpy()
print("Cosine similarity between 'light' embeddings in the two sentences:", cosine_similarity)

## Model Architecture

In [ ]:
from tensorflow.keras.utils import plot_model

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(10, 128)),
    LSTM(64),
    Dense(10, activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
plot_model(model, show_shapes=True, show_layer_names=True)


## Effect of temperature

In [ ]:
logits = np.array([2, 1, 0.1, 5, 3])
labels = ['mat' , 'tree', 'ball', 'bed', 'table']
temperatures = [0.5, 1, 2, 4]

def softmax_temperature_adjusted(logits, temperature):
    exp_logits = np.exp(logits / temperature)
    return exp_logits / np.sum(exp_logits)
    
plt.figure(figsize=(12, 8))
for T in temperatures:
    probabilities = softmax_temperature_adjusted(logits, T)
    plt.plot(labels, probabilities, marker='o', label=f'Temperature = {T}')
plt.title('Effect of Temperature on Prediction Distribution')
plt.ylabel('Probability')
plt.xlabel('Words')
plt.legend()
plt.grid(True)
plt.show()

## Zero-Shot Learning Setup

### Huggingface

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')
prompt = "Write a compelling product description for eco-friendly kitchenware emphasizing sustainability:"
completion = generator(prompt, max_length=100, num_return_sequences=1, temperature=0.7)
print("Generated text using HuggingFace's GPT model:", completion[0]['generated_text'])

### OpenAI API 

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = 'XXX') #insert your api key here 

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user",  "content": "Write a compelling product description for eco-friendly kitchenware emphasizing sustainability:"}],
    max_tokens=100, n=1, temperature=0.7)

print(completion.choices[0].message.content)

# Creating an Effective Prompt

In [ ]:
def generate_response(prompt, model="gpt-4", max_tokens=100, temperature=0.7, n=1):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=n,
        temperature=temperature)

    return response.choices[0].message.content

## Product descriptions

In [ ]:
good_product_description = "Write a captivating description for a bamboo cutlery set designed for eco-conscious consumers, emphasizing its sustainability and style."
generate_response(good_product_description)

In [ ]:
poor_product_description = "Talk about bamboo cutlery."
generate_response(poor_product_description)

## Blog Post Titles

In [ ]:
good_blog_title = "Create an intriguing title for a blog post highlighting the top five benefits of biodegradable kitchenware for sustainable living."
generate_response(good_blog_title)

In [ ]:
poor_blog_title = "Write a title about kitchenware benefits."
generate_response(poor_blog_title)

## Social Media Captions

In [ ]:
good_social_media_caption = "Create an engaging and witty Instagram caption for our latest eco-friendly kitchenware line, focusing on reducing plastic waste."
generate_response(good_social_media_caption)

In [ ]:
poor_social_media_caption = "Make an Instagram post for kitchenware."
generate_response(poor_social_media_caption)

# Parameter Tuning

In [ ]:
def generate_slogan(prompt, temperature=1.0, top_p=1.0):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=15,
        temperature=temperature,
        top_p=top_p,
        n=3
    )
    return (response.choices[0].message.content, response.choices[1].message.content, response.choices[2].message.content)

In [ ]:
prompt = "Create an impactful slogan for an eco-friendly clothing line that emphasizes sustainability and fashion."

In [ ]:
# Base case
generate_slogan(prompt)

## Temperature

In [ ]:
# High temperature
generate_slogan(prompt, temperature=1.8)

In [ ]:
# Low temperature
generate_slogan(prompt, temperature=0.3)

## Top P

In [ ]:
# Lowering Top P
generate_slogan(prompt, top_p=0.4)